# LLM Based Transformer for Text Summerization

In [19]:
import torch
import torch.nn as nn
import math

class LayerNormalization(nn.Module):

    def __init__(self, features: int, eps:float=10**-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features)) # alpha is a learnable parameter
        self.bias = nn.Parameter(torch.zeros(features)) # bias is a learnable parameter

    def forward(self, x):
        # x: (batch, seq_len, hidden_size)
         # Keep the dimension for broadcasting
        mean = x.mean(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # Keep the dimension for broadcasting
        std = x.std(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # eps is to prevent dividing by zero or when std is very small
        return self.alpha * (x - mean) / (std + self.eps) + self.bias

class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # w1 and b1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) # w2 and b2

    def forward(self, x):
        # (batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

class InputEmbeddings(nn.Module):

    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        # (batch, seq_len) --> (batch, seq_len, d_model)
        # Multiply by sqrt(d_model) to scale the embeddings according to the paper
        return self.embedding(x) * math.sqrt(self.d_model)
    
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        # Create a matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        # Create a vector of shape (seq_len)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # (seq_len, 1)
        # Create a vector of shape (d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model / 2)
        # Apply sine to even indices
        pe[:, 0::2] = torch.sin(position * div_term) # sin(position * (10000 ** (2i / d_model))
        # Apply cosine to odd indices
        pe[:, 1::2] = torch.cos(position * div_term) # cos(position * (10000 ** (2i / d_model))
        # Add a batch dimension to the positional encoding
        pe = pe.unsqueeze(0) # (1, seq_len, d_model)
        # Register the positional encoding as a buffer
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq_len, d_model)
        return self.dropout(x)

class ResidualConnection(nn.Module):
    
        def __init__(self, features: int, dropout: float) -> None:
            super().__init__()
            self.dropout = nn.Dropout(dropout)
            self.norm = LayerNormalization(features)
    
        def forward(self, x, sublayer):
            return x + self.dropout(sublayer(self.norm(x)))

class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model # Embedding vector size
        self.h = h # Number of heads
        # Make sure d_model is divisible by h
        assert d_model % h == 0, "d_model is not divisible by h"

        self.d_k = d_model // h # Dimension of vector seen by each head
        self.w_q = nn.Linear(d_model, d_model, bias=False) # Wq
        self.w_k = nn.Linear(d_model, d_model, bias=False) # Wk
        self.w_v = nn.Linear(d_model, d_model, bias=False) # Wv
        self.w_o = nn.Linear(d_model, d_model, bias=False) # Wo
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1]
        # Just apply the formula from the paper
        # (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            # Write a very low value (indicating -inf) to the positions where mask == 0
            attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len) # Apply softmax
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        # (batch, h, seq_len, seq_len) --> (batch, h, seq_len, d_k)
        # return attention scores which can be used for visualization
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        query = self.w_q(q) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        key = self.w_k(k) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        value = self.w_v(v) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)

        # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        # Calculate attention
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        
        # Combine all the heads together
        # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

        # Multiply by Wo
        # (batch, seq_len, d_model) --> (batch, seq_len, d_model)  
        return self.w_o(x)

class EncoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

    def forward(self, x, src_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x
    
class Encoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class DecoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x
    
class Decoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)

class ProjectionLayer(nn.Module):

    def __init__(self, d_model, vocab_size) -> None:
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x) -> None:
        # (batch, seq_len, d_model) --> (batch, seq_len, vocab_size)
        return self.proj(x)
    
class Transformer(nn.Module):

    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        # (batch, seq_len, d_model)
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)
    
    def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor):
        # (batch, seq_len, d_model)
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)
    
    def project(self, x):
        # (batch, seq_len, vocab_size)
        return self.projection_layer(x)
    
def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int=512, N: int=6, h: int=8, dropout: float=0.1, d_ff: int=2048) -> Transformer:
    # Create the embedding layers
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    # Create the positional encoding layers
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)
    
    # Create the encoder blocks
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    # Create the decoder blocks
    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)
    
    # Create the encoder and decoder
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))
    
    # Create the projection layer
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)
    
    # Create the transformer
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)
    
    # Initialize the parameters
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    
    return transformer

# Dataset

## Build Tokenizer

In [20]:
from datasets import load_dataset

path = "/kaggle/input/samsum-dataset-text-summarization/samsum-"
data_files = {
    "train": path+"train.csv",
    "validation": path+"validation.csv",
    "test": path + "test.csv" }
ds = load_dataset("csv", data_files=data_files)

ds = ds.map(lambda example: {
    "dialogue": str(example["dialogue"]),
    "summary": str(example["summary"])
})


In [21]:
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

from pathlib import Path

def get_all_sentences(ds, col):
    for item in ds:
        yield str(item[col])


def get_or_build_tokenizer(ds, col):
    tokenizer_path = Path('tokenizer_file_{}.json'.format(col))
    if not tokenizer_path.exists():
        # Most code taken from: https://huggingface.co/docs/tokenizers/quicktour
        tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)
        tokenizer.train_from_iterator(get_all_sentences(ds, col), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    return tokenizer

## Summerization Dataset

In [22]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

class SumerizationDataset(Dataset):

    def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_col, tgt_col, seq_len):
        super().__init__()
        self.seq_len = seq_len

        self.ds = ds
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.src_lang = src_col
        self.tgt_lang = tgt_col

        self.sos_token = torch.tensor([tokenizer_tgt.token_to_id("[SOS]")], dtype=torch.int64)
        self.eos_token = torch.tensor([tokenizer_tgt.token_to_id("[EOS]")], dtype=torch.int64)
        self.pad_token = torch.tensor([tokenizer_tgt.token_to_id("[PAD]")], dtype=torch.int64)

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        src_target_pair = self.ds[idx]
        src_text = src_target_pair[self.src_lang]
        tgt_text = src_target_pair[self.tgt_lang]

        # Transform the text into tokens
        enc_input_tokens = self.tokenizer_src.encode(src_text).ids
        dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids

          # Truncate the tokens if they exceed the sequence length
        enc_input_tokens = enc_input_tokens[:self.seq_len - 2]  # Reserve space for <s> and </s>
        dec_input_tokens = dec_input_tokens[:self.seq_len - 1]  # Reserve space for <s>


        # Add sos, eos and padding to each sentence
        enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2  # We will add <s> and </s>
        # We will only add <s>, and </s> only on the label
        dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1

        # Make sure the number of padding tokens is not negative. If it is, the sentence is too long
        if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
            raise ValueError("Sentence is too long")

        # Add <s> and </s> token
        encoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(enc_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        # Add only <s> token
        decoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        # Add only </s> token
        label = torch.cat(
            [
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        # Double check the size of the tensors to make sure they are all seq_len long
        assert encoder_input.size(0) == self.seq_len
        assert decoder_input.size(0) == self.seq_len
        assert label.size(0) == self.seq_len

        return {
            "encoder_input": encoder_input,  # (seq_len)
            "decoder_input": decoder_input,  # (seq_len)
            "encoder_mask": (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(), # (1, 1, seq_len)
            "decoder_mask": (decoder_input != self.pad_token).unsqueeze(0).int() & causal_mask(decoder_input.size(0)), # (1, seq_len) & (1, seq_len, seq_len),
            "label": label,  # (seq_len)
            "src_text": src_text,
            "tgt_text": tgt_text,
        }
    
def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0

## Tokenize data

In [23]:
tokenizer_src = get_or_build_tokenizer(ds['train'],'dialogue')
tokenizer_tgt = get_or_build_tokenizer(ds['train'],'summary')

In [24]:
max_len_src = 0
max_len_tgt = 0

for item in ds['train']:
        src_ids = tokenizer_src.encode(str(item['dialogue'])).ids
        tgt_ids = tokenizer_tgt.encode(str(item['summary'])).ids
        max_len_src = max(max_len_src, len(src_ids))
        max_len_tgt = max(max_len_tgt, len(tgt_ids))
    

In [25]:
seq_len = 128

In [26]:
train_ds = SumerizationDataset(ds['train'], tokenizer_src, tokenizer_tgt, 'dialogue', 'summary', seq_len)
val_ds = SumerizationDataset(ds['validation'], tokenizer_src, tokenizer_tgt, 'dialogue','summary', seq_len)
test_ds = SumerizationDataset(ds['test'], tokenizer_src, tokenizer_tgt, 'dialogue','summary', seq_len)

In [27]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=1, shuffle=True)

In [28]:
import torchmetrics
from torch.utils.tensorboard import SummaryWriter

def greedy_decode(model, source, source_mask, tokenizer_src, tokenizer_tgt, max_len, device):
    sos_idx = tokenizer_tgt.token_to_id('[SOS]')
    eos_idx = tokenizer_tgt.token_to_id('[EOS]')

    # Precompute the encoder output and reuse it for every step
    encoder_output = model.encode(source, source_mask)
    # Initialize the decoder input with the sos token
    decoder_input = torch.empty(1, 1).fill_(sos_idx).type_as(source).to(device)
    while True:
        if decoder_input.size(1) == max_len:
            break

        # build mask for target
        decoder_mask = causal_mask(decoder_input.size(1)).type_as(source_mask).to(device)

        # calculate output
        out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)

        # get next token
        prob = model.project(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat(
            [decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1
        )

        if next_word == eos_idx:
            break

    return decoder_input.squeeze(0)

In [29]:
def run_validation(model, validation_ds, tokenizer_src, tokenizer_tgt, max_len, device, print_msg, global_step, writer, num_examples=2):
    model.eval()
    count = 0

    source_texts = []
    expected = []
    predicted = []

    try:
        # get the console window width
        with os.popen('stty size', 'r') as console:
            _, console_width = console.read().split()
            console_width = int(console_width)
    except:
        # If we can't get the console width, use 80 as default
        console_width = 80

    with torch.no_grad():
        for batch in validation_ds:
            count += 1
            encoder_input = batch["encoder_input"].to(device) # (b, seq_len)
            encoder_mask = batch["encoder_mask"].to(device) # (b, 1, 1, seq_len)

            # check that the batch size is 1
            assert encoder_input.size(
                0) == 1, "Batch size must be 1 for validation"

            model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, 128, device)

            source_text = batch["src_text"][0]
            target_text = batch["tgt_text"][0]
            model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())

            source_texts.append(source_text)
            expected.append(target_text)
            predicted.append(model_out_text)
            
            # Print the source, target and model output
            print_msg('-'*console_width)
            print_msg(f"{f'SOURCE: ':>12}{source_text}")
            print_msg(f"{f'TARGET: ':>12}{target_text}")
            print_msg(f"{f'PREDICTED: ':>12}{model_out_text}")

            if count == num_examples:
                print_msg('-'*console_width)
                break
    
    if writer:
        # Compute the char error rate 
        metric = torchmetrics.text.CharErrorRate()
        cer = metric(predicted, expected)
        writer.add_scalar('validation cer', cer, global_step)
        writer.flush()

        # Compute the word error rate
        metric = torchmetrics.text.WordErrorRate()
        wer = metric(predicted, expected)
        writer.add_scalar('validation wer', wer, global_step)
        writer.flush()

        # Compute the BLEU metric
        metric = torchmetrics.text.BLEUScore()
        bleu = metric(predicted, expected)
        writer.add_scalar('validation BLEU', bleu, global_step)
        writer.flush()

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device

device(type='cuda')

## Build Transformer

In [31]:
 model = build_transformer(tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size(), seq_len, seq_len)

## Adam Optimizer

In [32]:
optimizer = torch.optim.Adam(model.parameters(), lr=10**-4, eps=1e-9)

## Cross entropy loss

In [33]:
loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1).to(device)


In [34]:
import torch
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import os


# TensorBoard writer
writer = SummaryWriter('runs/tmodel')

# Initialize device and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.nn.DataParallel(model, device_ids = [0,1])  # Wrap your model for multi-GPU training
model.to(device)

# Training loop setup
initial_epoch = 0
num_epochs = 40
global_step = 0
torch.cuda.empty_cache()

for epoch in range(initial_epoch, num_epochs):
    torch.cuda.empty_cache()
    model.train()
    batch_iterator = tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}")
    
    for batch in batch_iterator:
        encoder_input = batch['encoder_input'].to(device)  # Move inputs to GPUs
        decoder_input = batch['decoder_input'].to(device)
        encoder_mask = batch['encoder_mask'].to(device)
        decoder_mask = batch['decoder_mask'].to(device)

        # Forward pass
        encoder_output = model.module.encode(encoder_input, encoder_mask)  # Use `.module` when wrapped with DataParallel
        decoder_output = model.module.decode(encoder_output, encoder_mask, decoder_input, decoder_mask)
        proj_output = model.module.project(decoder_output)

        # Compare output with labels
        label = batch['label'].to(device)
        loss = loss_fn(proj_output.view(-1, tokenizer_tgt.get_vocab_size()), label.view(-1))
        batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}"})

        # Log loss
        writer.add_scalar('train loss', loss.item(), global_step)
        writer.flush()

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)

        global_step += 1

    # Validation
    run_validation(model.module, val_dataloader, tokenizer_src, tokenizer_tgt, 128, device,
                   lambda msg: batch_iterator.write(msg), global_step, writer)
    if epoch == num_epochs - 1:
        # Save model checkpoint
        model_filename =  f"model_epoch_{epoch:02d}.pt"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.module.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'global_step': global_step
        }, model_filename)

Processing Epoch 00: 100%|██████████| 461/461 [03:45<00:00,  2.05it/s, loss=5.359]
stty: 'standard input': Inappropriate ioctl for device


--------------------------------------------------------------------------------
    SOURCE: Keith: We have a few servings of fish extra from this morning if you want to join and chip in let me know, first come first serve
Alex: I WANT IT
Keith: I know
Keith: But you're here alrady, I'm asking the others
Alex: I'LL EAT EVERYTHING
Keith: Shut up!
Alex: NO
Keith: No more fish for ya
    TARGET: Alex wants to eat all extra fish but Keith refuses him.
 PREDICTED: is going to buy a .
--------------------------------------------------------------------------------
    SOURCE: Elena: Have you read about the British teenager that escaped to Syria in 2015?
Kim: no, but I remember the case
Elena: <file_other>
Andreas: there was a similar case in Austria at the beginning of the war
Andreas: but, honestly, I cannot relate to their problems
Andreas: so much naivety and stupidity
Jeff: So she wants to come back now?
Elena: apparently
Elena: but she also have no regrets
Kim: I think it's dangerous
El

Processing Epoch 01: 100%|██████████| 461/461 [03:54<00:00,  1.97it/s, loss=5.171]
stty: 'standard input': Inappropriate ioctl for device


--------------------------------------------------------------------------------
    SOURCE: Ben: Yo?
Jake: Yo.
Ben: Where you at?
Jake: Out.
Ben: Yeah I need your laptop.
Jake: What for?
Ben: Gotta do some work stuff.
Jake: What's with yours?
Ben: It's having that adapter issue, remember?
Jake: Oh. Yeah sure just take it from my room.
Ben: Yeah thanks. And by the way, you're not gonna get it till tomorrow.
Jake: Okay man.
Ben: So you won't get to play DOTA tonight.
Jake: That's fine.
Ben: Okay. Great!
    TARGET: Ben needs Jakes laptop as his has some adapter issues. He will return it tomorrow, so Jake won't be able to play DOTA tonight.
 PREDICTED: is going to the of the . She will be in the .
--------------------------------------------------------------------------------
    SOURCE: Frederic: how is the weather down there?
Sean: today it's sunny but cold
Vanessa: it's quite strange, the temperature is low but it seems quite hot in the sun 
Abigail: I love this weather, there is no 

Processing Epoch 02: 100%|██████████| 461/461 [03:53<00:00,  1.97it/s, loss=5.038]
stty: 'standard input': Inappropriate ioctl for device


...

--------------------------------------------------------------------------------
    SOURCE: Jim: is it forest school tomorrow?
Nia: only for group 3 and 4, Lindsay is in group 2
Jim: thanks
Sadie: Jim, Elsa is in group 2 too
    TARGET: Forest school is tomorrow for group 3 and 4. Lindsay and Elsa are in group 2.
 PREDICTED: Nia ' s teacher , and Jason are going to school tomorrow .
--------------------------------------------------------------------------------
    SOURCE: Grayson: Hey, dad! :) Could you give me a lift to the airport?
Harrison: when?
Grayson: Tomorrow.  I have to check in two hours before the flight, so I should be in the airport around 4 pm.
Harrison: i'm at work at this time
Harrison: ask your mother
Harrison: where are you flying to? for how long?
Grayson: I'm going to Spain for 2 weeks. :)
Harrison: have fun and don't forget to send me a postcard :-]
Harrison: don't drink too much
Grayson: Thanks, dad, I won't! :)
    TARGET: Grayson is going to Spain for two wee

Processing Epoch 38: 100%|██████████| 461/461 [03:52<00:00,  1.98it/s, loss=1.608]
stty: 'standard input': Inappropriate ioctl for device


--------------------------------------------------------------------------------
    SOURCE: Peter: Hi! How was your work?
Amy: Hi. Well, the day was weird.
Peter: Y?
Amy: Managing a team is hard!
Peter: I know. What happened?
Amy: Well, there's this Mark. And his attitude is simply horrible.
Peter: What did he do?
Amy: Actually nothing much, but it's the general picture that matters.
Peter: What do you mean?
Amy: That's a long story.
Peter: Don't worry. Got time.
    TARGET: Amy had a weird day at work managing her team. Amy dislikes the attitude of one of her coworkers, Mark.
 PREDICTED: Amy ' s company got into and he ' s really into the team . Peter is worried about the team that the team will probably be difficult to talk about this time .
--------------------------------------------------------------------------------
    SOURCE: Charles: Hi! Would you like to go to the opera next month.
Carol: Anything in particular?
Charles: Carmen
Carol: I've always wanted to see it! Let's go


Processing Epoch 39: 100%|██████████| 461/461 [03:53<00:00,  1.98it/s, loss=1.522]
stty: 'standard input': Inappropriate ioctl for device


--------------------------------------------------------------------------------
    SOURCE: Guto: Guys, be careful when talking to Richard
Guto: he's not with Peter anymore
Stephen: oh no, what happened?
Ross: I know, he mentioned it
Ross: but I've no idea why
Han: very unexpected
Ross: he said something like: "Peter didn't like who I am"
Han: sad, sounds serious
Stephen: yes, like this realisation you're not accepted by your partner
Guto: so maybe it's good they broke up
Ross: Which does not change the fact that it may be very difficult for them
Guto: how long did they stay together?
Ross: 3 years at least
Guto: right, a piece of life is gone...
    TARGET: Richard and Peter broke up. They were more than 3 years together. Peter didn't accept Richard fully. Stephen, Guto, Ross and Han will be careful when talking to Richard.
 PREDICTED: After graduating , Peter is upset , because he needs to do her .
--------------------------------------------------------------------------------
    

## Evaluate

In [60]:
import torch
from torchmetrics.text.rouge import ROUGEScore
from pprint import pprint

def run_test(model, test_dataloader, tokenizer_src, tokenizer_tgt, max_len, device):
    model.eval()

    source_texts = []
    expected = []
    predicted = []

    console_width = 80

    # Initialize ROUGEScore to compute only ROUGE-1, ROUGE-2, and ROUGE-L
    rouge = ROUGEScore(rouge_keys=('rouge1', 'rouge2', 'rougeL')).to(device)
    all_rouge_scores = []
    count = 0
    with torch.no_grad():
        for batch in test_dataloader:
            count += 1
            encoder_input = batch["encoder_input"].to(device)  # (b, seq_len)
            encoder_mask = batch["encoder_mask"].to(device)  # (b, 1, 1, seq_len)

            # Check that the batch size is 1
            assert encoder_input.size(0) == 1, "Batch size must be 1 for test"

            model_out = greedy_decode(model, encoder_input, encoder_mask, tokenizer_src, tokenizer_tgt, max_len, device)

            source_text = batch["src_text"][0]
            target_text = batch["tgt_text"][0]
            model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy())

            source_texts.append(source_text)
            expected.append(target_text)
            predicted.append(model_out_text)

            
            

            # Calculate and print ROUGE scores for this example
            rouge_scores = rouge(model_out_text, target_text)
            all_rouge_scores.append(rouge_scores)
            if count % 50 == 0:
                # Print the source, target and model output
                pprint('-' * console_width)
                pprint("ROUGE scores for this example:")
                pprint({k: v.item() for k, v in rouge_scores.items()})

    if all_rouge_scores:
        # Calculate and print average ROUGE scores
        avg_rouge_scores = {key: torch.mean(torch.stack([score[key] for score in all_rouge_scores])).item() for key in all_rouge_scores[0]}
        pprint("Average ROUGE scores:")
        pprint(avg_rouge_scores)
    else:
        pprint("No ROUGE scores were calculated. Please check the test_dataloader and ensure it contains data.")

# Example usage
run_test(model.module, test_dataloader, tokenizer_src, tokenizer_tgt, 128, device)

'--------------------------------------------------------------------------------'
'ROUGE scores for this example:'
{'rouge1_fmeasure': 0.31111112236976624,
 'rouge1_precision': 0.46666666865348816,
 'rouge1_recall': 0.23333333432674408,
 'rouge2_fmeasure': 0.09302325546741486,
 'rouge2_precision': 0.1428571492433548,
 'rouge2_recall': 0.06896551698446274,
 'rougeL_fmeasure': 0.2222222238779068,
 'rougeL_precision': 0.3333333432674408,
 'rougeL_recall': 0.1666666716337204}
'--------------------------------------------------------------------------------'
'ROUGE scores for this example:'
{'rouge1_fmeasure': 0.20000000298023224,
 'rouge1_precision': 0.29411765933036804,
 'rouge1_recall': 0.1515151560306549,
 'rouge2_fmeasure': 0.0,
 'rouge2_precision': 0.0,
 'rouge2_recall': 0.0,
 'rougeL_fmeasure': 0.11999999731779099,
 'rougeL_precision': 0.1764705926179886,
 'rougeL_recall': 0.09090909361839294}
'--------------------------------------------------------------------------------'
'ROUGE 

In [66]:
import torch
from pathlib import Path
from tokenizers import Tokenizer
from datasets import load_dataset

def summarize(sentence: str, model_path: str, tokenizer_src_path: str, tokenizer_tgt_path: str):
    
    # Define the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Load tokenizers
    tokenizer_src = Tokenizer.from_file(tokenizer_src_path)
    tokenizer_tgt = Tokenizer.from_file(tokenizer_tgt_path)
    
    # Load model
    model = build_transformer(tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size(), 128, 128).to(device)
    
    # Load the pretrained weights
    state = torch.load(model_path,weights_only=False)
    model.load_state_dict(state['model_state_dict'])
    
    # Translate the sentence
    model.eval()
    with torch.no_grad():
        # Precompute the encoder output and reuse it for every generation step
        source = tokenizer_src.encode(sentence)
        source = torch.cat([
            torch.tensor([tokenizer_src.token_to_id('[SOS]')], dtype=torch.int64), 
            torch.tensor(source.ids, dtype=torch.int64),
            torch.tensor([tokenizer_src.token_to_id('[EOS]')], dtype=torch.int64),
            torch.tensor([tokenizer_src.token_to_id('[PAD]')] * (seq_len - len(source.ids) - 2), dtype=torch.int64)
        ], dim=0).to(device)
        source_mask = (source != tokenizer_src.token_to_id('[PAD]')).unsqueeze(0).unsqueeze(0).int().to(device)
        encoder_output = model.encode(source, source_mask)
        
        # Initialize the decoder input with the sos token
        decoder_input = torch.empty(1, 1).fill_(tokenizer_tgt.token_to_id('[SOS]')).type_as(source).to(device)
        
        # Generate the translation word by word
        while decoder_input.size(1) < seq_len:
            # Build mask for target and calculate output
            decoder_mask = torch.triu(torch.ones((1, decoder_input.size(1), decoder_input.size(1))), diagonal=1).type(torch.int).type_as(source_mask).to(device)
            out = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)
            
            # Project next token
            prob = model.project(out[:, -1])
            _, next_word = torch.max(prob, dim=1)
            decoder_input = torch.cat([decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1)
            
            # Break if we predict the end of sentence token
            if next_word == tokenizer_tgt.token_to_id('[EOS]'):
                break
    
    # Convert ids to tokens
    return tokenizer_tgt.decode(decoder_input[0].tolist())


model_path = '/kaggle/working/model_epoch_39.pt'
tokenizer_src_path = '/kaggle/working/tokenizer_file_dialogue.json'
tokenizer_tgt_path = '/kaggle/working/tokenizer_file_summary.json'


Summarized Text: recommends the Formula Formula Formula 1 with the cheese that for the Formula .


## Test 1

In [67]:
sentence = """
Rose: I'm going to the airport
Violet: Have a safe flight!
Rose: Thanks! See you in a few days.
"""
summarized_text = summarize(sentence, model_path, tokenizer_src_path, tokenizer_tgt_path)
print("Summarized Text:", summarized_text)

Summarized Text: Rose is going to the airport in a long - already here is going to see Rose days in a few days in a few days .


# Test 2

In [68]:
sentence = """
 Adam: It's so boring here….
Mandy: Still at class?
Adam: Yes.
Adam: -_-
"""
summarized_text = summarize(sentence, model_path, tokenizer_src_path, tokenizer_tgt_path)
print("Summarized Text:", summarized_text)

Summarized Text: Adam is still at class is still at class .
